# Case Study по модулю SQL
**Исполнитель:** [Behzod Jumaev]  
**Дата исполнения:** [13.11.2024]

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from connector import connect_to, read_credentials
from sqlalchemy import text, MetaData, Table, Column, Integer,\
                       String, Date, Numeric, ForeignKey, create_engine


## Блок 1. Создание схемы и таблиц  
1. В базе данных создайте новую схему `adv_works`.  
2. На основе данных из [файла](https://github.com/datacoach2024/data_analysis_course/blob/main/data/adventure_works.xlsx) создайте в схеме `adv_works` таблицы и соедините их между собою по основным и внешним ключам.

In [47]:
query_schema = '''
create schema if not exists adv_works;
set search_path to adv_works;'''

query_customer = '''
create table if not exists customers (
    customer_key int primary key,
    geography_key int,
    name varchar(100),
    birth_date date,
    marital_status char(1),
    gender char(1),
    yearly_income decimal(10, 2),
    number_children_at_home int,
    occupation varchar(50),
    house_owner_flag bit,
    number_cars_owned int,
    address_line1 varchar(100),
    address_line2 varchar(100),
    phone varchar(20),
    date_first_purchase date
);'''

query_territory = '''
create table if not exists territory (
    territory_key int primary key,
    region varchar(50),
    country varchar(50),
    group_name varchar(50)
);'''

query_product_category = '''
create table if not exists product_category (
    product_category_key int primary key,
    product_category_alternate_key int,
    english_product_category_name varchar(50),
    spanish_product_category_name varchar(50),
    french_product_category_name varchar(50)
);'''

query_product_subcategory = '''
create table if not exists product_subcategory (
    product_subcategory_key int primary key,
    product_subcategory_alternate_key int,
    english_product_subcategory_name varchar(50),
    spanish_product_subcategory_name varchar(50),
    french_product_subcategory_name varchar(50),
    product_category_key int
);'''

query_product = '''
create table if not exists products (
    product_key int primary key,
    product_subcategory_key int,
    product_name varchar(100),
    standard_cost decimal(18, 4),
    color varchar(50),
    safety_stock_level int,
    list_price decimal(18, 4),
    size varchar(20),
    size_range varchar(20),
    weight decimal(10, 2),
    days_to_manufacture int,
    product_line char(2),
    dealer_price decimal(18, 4),
    class char(1),
    model_name varchar(50),
    description text,
    start_date date,
    end_date date,
    status varchar(20)
);'''

query_sale = '''
create table if not exists sales (
    product_key int,
    order_date date,
    order_date_key int,
    customer_key int,
    sales_territory_key int,
    sales_order_number varchar(20),
    sales_order_line_number int,
    order_quantity int,
    unit_price decimal(18, 4),
    extended_amount decimal(18, 4),
    unit_price_discount_pct decimal(5, 2),
    discount_amount decimal(18, 4),
    product_standard_cost decimal(18, 4),
    total_product_cost decimal(18, 4),
    sales_amount decimal(18, 4),
    tax_amt decimal(18, 4),
    freight decimal(18, 4),
    region_month_id varchar(50)
);
commit;'''

query_foreign_key = '''
alter table product_subcategory
    add constraint fk_product_category
    foreign key (product_category_key) references product_category(product_category_key);

alter table products
    add constraint fk_product_subcategory
    foreign key (product_subcategory_key) references product_subcategory(product_subcategory_key);

alter table sales
    add constraint fk_sales_product
    foreign key (product_key) references products(product_key);

alter table sales
    add constraint fk_sales_customer
    foreign key (customer_key) references customers(customer_key);

alter table sales
    add constraint fk_sales_territory
    foreign key (sales_territory_key) references territory(territory_key);
commit;'''

In [58]:
with connect_to() as conn:
    conn.execute(text(query_schema))
    conn.execute(text(query_customer))
    conn.execute(text(query_territory))
    conn.execute(text(query_product_category))
    conn.execute(text(query_product_subcategory))
    conn.execute(text(query_product))
    conn.execute(text(query_sale))
    conn.commit()
    conn.execute(text(query_foreign_key))
    conn.commit()

In [63]:
file_path = 'adventure_works.xlsx'
excel_data = pd.ExcelFile(file_path)

def preprocess_customers(df):
    df = df.rename(columns={
        'CustomerKey': 'customer_key',
        'GeographyKey': 'geography_key',
        'Name': 'name',
        'BirthDate': 'birth_date',
        'MaritalStatus': 'marital_status',
        'Gender': 'gender',
        'YearlyIncome': 'yearly_income',
        'NumberChildrenAtHome': 'number_children_at_home',
        'Occupation': 'occupation',
        'HouseOwnerFlag': 'house_owner_flag',
        'NumberCarsOwned': 'number_cars_owned',
        'AddressLine1': 'address_line1',
        'AddressLine2': 'address_line2',
        'Phone': 'phone',
        'DateFirstPurchase': 'date_first_purchase'
    })
    df['house_owner_flag'] = df['house_owner_flag'].apply(lambda x: '1' if x else '0')
    return df

def preprocess_territory(df):
    df = df.rename(columns={
        'Territory Key': 'territory_key',
        'Region': 'region',
        'Country': 'country',
        'Group': 'group_name'
    })
    return df

def preprocess_product_category(df):
    df = df.rename(columns={
        'ProductCategoryKey': 'product_category_key',
        'ProductCategoryAlternateKey': 'product_category_alternate_key',
        'EnglishProductCategoryName': 'english_product_category_name',
        'SpanishProductCategoryName': 'spanish_product_category_name',
        'FrenchProductCategoryName': 'french_product_category_name'
    })
    return df

def preprocess_product_subcategory(df):
    df = df.rename(columns={
        'ProductSubcategoryKey': 'product_subcategory_key',
        'ProductSubcategoryAlternateKey': 'product_subcategory_alternate_key',
        'EnglishProductSubcategoryName': 'english_product_subcategory_name',
        'SpanishProductSubcategoryName': 'spanish_product_subcategory_name',
        'FrenchProductSubcategoryName': 'french_product_subcategory_name',
        'ProductCategoryKey': 'product_category_key'
    })
    return df

def preprocess_products(df):
    df = df.rename(columns={
        'ProductKey': 'product_key',
        'ProductSubcategoryKey': 'product_subcategory_key',
        'ProductName': 'product_name',
        'StandardCost': 'standard_cost',
        'Color': 'color',
        'SafetyStockLevel': 'safety_stock_level',
        'ListPrice': 'list_price',
        'Size': 'size',
        'SizeRange': 'size_range',
        'Weight': 'weight',
        'DaysToManufacture': 'days_to_manufacture',
        'ProductLine': 'product_line',
        'DealerPrice': 'dealer_price',
        'Class': 'class',
        'ModelName': 'model_name',
        'Description': 'description',
        'StartDate': 'start_date',
        'EndDate': 'end_date',
        'Status': 'status'
    })
    return df

def preprocess_sales(df):
    df = df.rename(columns={
        'ProductKey': 'product_key',
        'OrderDate': 'order_date',
        'OrderDateKey': 'order_date_key',
        'CustomerKey': 'customer_key',
        'SalesTerritoryKey': 'sales_territory_key',
        'SalesOrderNumber': 'sales_order_number',
        'SalesOrderLineNumber': 'sales_order_line_number',
        'OrderQuantity': 'order_quantity',
        'UnitPrice': 'unit_price',
        'ExtendedAmount': 'extended_amount',
        'UnitPriceDiscountPct': 'unit_price_discount_pct',
        'DiscountAmount': 'discount_amount',
        'ProductStandardCost': 'product_standard_cost',
        'TotalProductCost': 'total_product_cost',
        'SalesAmount': 'sales_amount',
        'TaxAmt': 'tax_amt',
        'Freight': 'freight',
        'RegionMonthID': 'region_month_id'
    })
    return df

df_customers = preprocess_customers(excel_data.parse('Customers'))
df_territory = preprocess_territory(excel_data.parse('Territory'))
df_product_category = preprocess_product_category(excel_data.parse('ProductCategory'))
df_product_subcategory = preprocess_product_subcategory(excel_data.parse('ProductSubCategory'))
df_products = preprocess_products(excel_data.parse('Products'))
df_sales = preprocess_sales(excel_data.parse('Sales'))


In [ ]:
with connect_to('postgres') as pg:
    pg.execute(text("set search_path to joins, adv_works"))
    pg.commit()

    df_customers.to_sql(
        name='customers',
        con=pg,
        index=False,
        if_exists='append'
    )
    df_territory.to_sql(
        name='territory',
        con=pg,
        index=False,
        if_exists='append'
    )
    df_product_category.to_sql(
        name='product_category',
        con=pg,
        index=False,
        if_exists='append'
    )
    df_product_subcategory.to_sql(
        name='product_subcategory',
        con=pg,
        index=False,
        if_exists='append'
    )
    df_products.to_sql(
        name='products',
        con=pg,
        index=False,
        if_exists='append'
    )
    df_sales.to_sql(
        name='sales',
        con=pg,
        index=False,
        if_exists='append'
    )
    
    pg.commit()

## Блок 2. Аналитические задачи  
### Секция 1. Анализ клиентов  
1. **Сегментация по доходу**: Посчитайте средний годовой личный доход клиентов (*YearlyIncome*) в разбивке по роду деятельности (*Occupation*). Итоговая таблица должна содержать следующие поля: *occupation*, *number_of_customers*, *avg_income*.  
2.  **Семейный профиль**: Посчитайте долю (в процентах) клиентов с детьми и долю клиентов без детей. Итоговая таблица должна содержать следующие поля: *has_children* (где 1 означает - имеет детей и 0 - не имеет детей), *pct_of_customer_base*.
3.  **Высокодоходные клиенты**: Сформируйте список топ 10 клиентов с наибольшей суммой покупок (поле `SalesAmount`). Итоговая таблица должна содержать следующие поля: *customer_key*, *customer_name*, *total_purchase*.
4.  **Влияние семейного положения**: Посчитайте среднюю сумму продаж в разбивке по семейному положению клиентов (*MaritalStatus*) и определите насколько сильно различаются средние суммы между двумя группами. Итоговая таблица должна содержать следующие поля: *year*, *marital_status*, *avg_sales_amount*.  

#### 1. Сегментация по доходу

In [127]:
query_income_segmentation = """
set search_path to adv_works;
select 
    occupation,
    count(*) as number_of_customers,
    round(avg(yearly_income), 2) as avg_income
from 
    customers
group by 
    occupation;
"""

with connect_to() as pg:
    df_income_segmentation = pd.read_sql(query_income_segmentation, pg)

df_income_segmentation

,occupation,number_of_customers,avg_income
0,Management,3075,92325.20
1,Clerical,2928,30710.38
2,Manual,2384,16451.34
3,Skilled Manual,4577,51715.10
4,Professional,5520,74184.78


#### 2. Семейный профиль

In [128]:
query_family_profile = """
set search_path to adv_works;
select 
    case 
        when number_children_at_home > 0 then 1 
        else 0 
    end as has_children,
    round(count(*) * 100.0 / (select count(*) from customers), 2) as pct_of_customer_base
from 
    customers
group by 
    has_children;
"""
with connect_to() as pg:
    df_family_profile = pd.read_sql(query_family_profile, pg)

df_family_profile


,has_children,pct_of_customer_base
0,0,60.14
1,1,39.86


#### 3. Высокодоходные клиенты


In [129]:
query_high_income_customers = """
set search_path to adv_works;
select 
    c.customer_key,
    c.name AS customer_name,
    round(sum(s.sales_amount), 2) as total_purchase
from 
    sales as s
join 
    customers as c on s.customer_key = c.customer_key
group by 
    c.customer_key, c.name
order by 
    total_purchase desc
limit 10;
"""
with connect_to() as pg:
    df_high_income_customers = pd.read_sql(query_high_income_customers, pg)

df_high_income_customers

,customer_key,customer_name,total_purchase
0,12301,Nichole Nara,13295.38
1,12132,Kaitlyn Henderson,13294.27
2,12308,Margaret He,13269.27
3,12131,Randall Dominguez,13265.99
4,12300,Adriana Gonzalez,13242.70
5,12321,Rosa Hu,13215.65
6,12124,Brandi Gill,13195.64
7,12307,Brad She,13173.19
8,12296,Francisco Sara,13164.64
9,11433,Maurice Shan,12909.67


#### 4. Влияние семейного положения

In [130]:
query_marital_status_impact = """
set search_path to adv_works;
select 
    extract(year from s.order_date) as year,
    c.marital_status,
    round(avg(s.sales_amount), 2) as avg_sales_amount
from 
    sales as s
join 
    customers as c on s.customer_key = c.customer_key
group by 
    year, c.marital_status
order by 
    year, c.marital_status;
"""

with connect_to() as pg:
    df_marital_status_impact = pd.read_sql(query_marital_status_impact, pg)
    df_marital_status_impact['year'] = df_marital_status_impact['year'].astype(int)

df_marital_status_impact

,year,marital_status,avg_sales_amount
0,2001,M,3245.03
1,2001,S,3203.84
2,2002,M,2397.07
3,2002,S,2482.13
4,2003,M,378.56
5,2003,S,427.78
6,2004,M,290.64
7,2004,S,318.05


### Секция 2. Анализ продаж  
 1. **Ежемесячные продажи**: Создайте отчёт продаж по месяцам за последние 2 года (2003, 2004). Итоговая таблица должна содержать следующие поля: *year*, *monthkey*, *month_name*, *sales_count* (количество продаж), *sales_amount*.  
 2. **Продажи по регионам**: Посчитайте сумму продаж в разбивке по регионам. Итоговая таблица должна содержать следующие поля: *region*, *sales_count*, *sales_amount*.  

#### 1. Ежемесячные продажи

In [131]:
query_monthly_sales = """
set search_path to adv_works;
select 
    extract(year from order_date) as year,
    extract(month from order_date) as monthkey,
    to_char(order_date, 'Month') as month_name,
    count(*) as sales_count,
    round(sum(sales_amount), 2) as sales_amount
from 
    sales
where 
    extract(year from order_date) in (2003, 2004)
group by 
    year, monthkey, month_name
order by 
    year, monthkey;
"""

with connect_to() as pg:
    df_monthly_sales = pd.read_sql(query_monthly_sales, pg)
df_monthly_sales


,year,monthkey,month_name,sales_count,sales_amount
0,2003.0,1.0,January,244,438865.17
1,2003.0,2.0,February,272,489090.34
2,2003.0,3.0,March,272,485574.79
3,2003.0,4.0,April,294,506399.27
4,2003.0,5.0,May,335,562772.56
5,2003.0,6.0,June,321,554799.23
6,2003.0,7.0,July,1411,886668.84
7,2003.0,8.0,August,3819,847413.51
8,2003.0,9.0,September,3885,1010258.13
9,2003.0,10.0,October,4146,1080449.58


#### 2. Продажи по регионам

In [132]:
query_sales_by_region = """
set search_path to adv_works;
select 
    t.region,
    count(s.sales_order_number) as sales_count,
    round(sum(s.sales_amount), 2) as sales_amount
from 
    sales as s
join 
    territory as t on s.sales_territory_key = t.territory_key
group by 
    t.region
order by 
    sales_amount desc;
"""

with connect_to() as pg:
    df_sales_by_region = pd.read_sql(query_sales_by_region, pg)
df_sales_by_region


,region,sales_count,sales_amount
0,Australia,13345,9061000.58
1,Southwest,12265,5718150.81
2,Northwest,8993,3649866.55
3,United Kingdom,6906,3391712.21
4,Germany,5625,2894312.34
5,France,5558,2644017.71
6,Canada,7620,1977844.86
7,Southeast,39,12238.85
8,Northeast,27,6532.47
9,Central,20,3000.83


### Секция 3. Анализ продуктов  
 1. **Доля продаж**: Посчитайте какую долю от общих продаж составляет каждая категория продуктов. Итоговая таблица должна содержать следующие поля: *year*, *product_key*, *product_category_key*, *english_product_category_name*, *sales_amount*, *pct_of_total_sales*.
 2. **Самые продаваемые продукты**: Определите топ 5 продуктов с наибольшей суммой продаж. Итоговая таблица должна содержать следующие поля: *product_key*, *product_name*, *english_product_category_name*, *sales_amount*
 3. **Маржа от продаж**: Посчитайте разницу между суммой продаж (*SalesAmount*) за минусом себестоимости (*TotalProductCost*), налогов (*TaxAmt*) и расходов на доставку (*Freight*) по каждому продукту в разбивке по годам и месяцам. Итоговая таблица должна содержать следующие поля: *year*, *monthkey*, *month_name*, *product_key*, *product_name*, *sales_amount*, *total_product_cost*, *tax_amt*, *freight*, *margin*, *margin_pct* (маржа как процент от суммы продаж).

#### 1. Доля продаж по категориям продуктов

In [154]:
query_sales_share_by_category = """
set search_path to adv_works;
with total_sales as (
    select 
        extract(year from s.order_date) as year,
        pc.product_category_key,
        pc.english_product_category_name,
        sum(s.sales_amount) as sales_amount
    from 
        sales as s
    join 
        products as p on s.product_key = p.product_key
    join 
        product_subcategory as ps on p.product_subcategory_key = ps.product_subcategory_key
    join 
        product_category as pc on ps.product_category_key = pc.product_category_key
    group by 
        year, pc.product_category_key, pc.english_product_category_name
),
overall_sales as (
    select 
        year,
        sum(sales_amount) as total_sales_amount
    from 
        total_sales
    group by 
        year
)
select 
    ts.year,
    p.product_key,
    ts.product_category_key,
    ts.english_product_category_name,
    ts.sales_amount,
    round((ts.sales_amount * 100.0 / os.total_sales_amount), 2) as pct_of_total_sales
from 
    total_sales as ts
join 
    overall_sales as os on ts.year = os.year
join 
    products as p on ts.product_category_key = (select product_category_key from product_subcategory where product_subcategory_key = p.product_subcategory_key)
order by 
    ts.year, ts.product_category_key;
"""

with connect_to() as pg:
    df_sales_share_by_category = pd.read_sql(query_sales_share_by_category, pg)
df_sales_share_by_category


,year,product_key,product_category_key,english_product_category_name,sales_amount,pct_of_total_sales
0,2001.0,364,1,Bikes,3.266374e+06,100.00
1,2001.0,331,1,Bikes,3.266374e+06,100.00
2,2001.0,586,1,Bikes,3.266374e+06,100.00
3,2001.0,378,1,Bikes,3.266374e+06,100.00
4,2001.0,585,1,Bikes,3.266374e+06,100.00
...,...,...,...,...,...,...
661,2004.0,483,4,Accessories,4.070502e+05,4.17
662,2004.0,220,4,Accessories,4.070502e+05,4.17
663,2004.0,452,4,Accessories,4.070502e+05,4.17
664,2004.0,536,4,Accessories,4.070502e+05,4.17


#### 2. Самые продаваемые продукты

In [151]:
query_top_selling_products = """
set search_path to adv_works;
select 
    p.product_key,
    p.product_name,
    pc.english_product_category_name,
    round(sum(s.sales_amount), 2) as sales_amount
from 
    sales as s
join 
    products as p on s.product_key = p.product_key
join 
    product_subcategory as ps on p.product_subcategory_key = ps.product_subcategory_key
join 
    product_category as pc on ps.product_category_key = pc.product_category_key
group by 
    p.product_key, p.product_name, pc.english_product_category_name
order by 
    sales_amount desc
limit 5;
"""

with connect_to() as pg:
    df_top_selling_products = pd.read_sql(query_top_selling_products, pg)
df_top_selling_products


,product_key,product_name,english_product_category_name,sales_amount
0,312,"Road-150 Red, 48",Bikes,1205876.99
1,310,"Road-150 Red, 62",Bikes,1202298.72
2,313,"Road-150 Red, 52",Bikes,1080637.54
3,314,"Road-150 Red, 56",Bikes,1055589.65
4,311,"Road-150 Red, 44",Bikes,1005493.87


#### 3. Маржа от продаж

In [153]:
query_sales_margin = """
set search_path to adv_works;
select 
    extract(year from s.order_date) as year,
    extract(month from s.order_date) as monthkey,
    to_char(s.order_date, 'FMMonth') as month_name,
    p.product_key,
    p.product_name,
    round(sum(s.sales_amount), 2) as sales_amount,
    round(sum(s.total_product_cost), 2) as total_product_cost,
    round(sum(s.tax_amt), 2) as tax_amt,
    round(sum(s.freight), 2) as freight,
    round(sum(s.sales_amount - s.total_product_cost - s.tax_amt - s.freight), 2) as margin,
    round((sum(s.sales_amount - s.total_product_cost - s.tax_amt - s.freight) * 100.0 / nullif(sum(s.sales_amount), 0)), 2) as margin_pct
from 
    sales as s
join 
    products as p on s.product_key = p.product_key
group by 
    year, monthkey, month_name, p.product_key, p.product_name
order by 
    year, monthkey, margin desc;
"""

with connect_to() as pg:
    df_sales_margin = pd.read_sql(query_sales_margin, pg)
    df_sales_margin['year'] = df_sales_margin['year'].astype(int)
df_sales_margin


,year,monthkey,month_name,product_key,product_name,sales_amount,total_product_cost,tax_amt,freight,margin,margin_pct
0,2001,7.0,July,312,"Road-150 Red, 48",100191.56,60796.24,8015.32,2504.79,28875.21,28.82
1,2001,7.0,July,311,"Road-150 Red, 44",82300.21,49939.77,6584.02,2057.51,23718.92,28.82
2,2001,7.0,July,310,"Road-150 Red, 62",78721.94,47768.47,6297.76,1968.05,22687.66,28.82
3,2001,7.0,July,314,"Road-150 Red, 56",53674.05,32569.41,4293.92,1341.85,15468.86,28.82
4,2001,7.0,July,313,"Road-150 Red, 52",42939.24,26055.53,3435.14,1073.48,12375.09,28.82
...,...,...,...,...,...,...,...,...,...,...,...
1895,2004,7.0,July,481,"Racing Socks, M",152.83,57.16,12.23,3.82,79.62,52.10
1896,2004,7.0,July,488,"Short-Sleeve Classic Jersey, S",593.89,457.30,47.51,14.85,74.24,12.50
1897,2004,7.0,July,490,"Short-Sleeve Classic Jersey, L",431.92,332.58,34.55,10.80,53.99,12.50
1898,2004,7.0,July,479,Road Bottle Cage,89.90,33.62,7.19,2.25,46.84,52.10


### Секция 4. Анализ трендов  
1. **Квартальный рост**: Посчитайте сумму продаж за каждый квартал и их процентное изменение по топ 2 наиболее продаваемым категориям. Итоговая таблица должна содержать следующие поля: *year*, *quarter_id*, *product_category_key*, *english_product_category_name*, *quarter_sales_amount*, *quarter_over_quarter_growth_pct*.
2. **Сравнение будних и выходных (суббота, воскресенье) дней**: Посчитайте продажи в разбивке по годам и дням недели. Определите в какие дни в среднем сумма продаж больше. Определите является ли сумма продаж больше в будние или выходные дни. Итоговая таблица должна содержать следующие поля: *year*, *day_name*, *is_weekend* (где 1 означает выходной а 0 будний день), *sales_amount*.

#### 1. **Квартальный рост**

In [163]:
query_quarterly_growth = """
set search_path to adv_works;
with quarterly_sales as (
    select 
        extract(year from s.order_date) as year,
        extract(quarter from s.order_date) as quarter_id,
        pc.product_category_key,
        pc.english_product_category_name,
        sum(s.sales_amount) as quarter_sales_amount
    from 
        sales as s
    join 
        products as p on s.product_key = p.product_key
    join 
        product_subcategory as ps on p.product_subcategory_key = ps.product_subcategory_key
    join 
        product_category as pc on ps.product_category_key = pc.product_category_key
    group by 
        year, quarter_id, pc.product_category_key, pc.english_product_category_name
),
top_categories as (
    select 
        product_category_key,
        english_product_category_name
    from 
        quarterly_sales
    group by 
        product_category_key, english_product_category_name
    order by 
        sum(quarter_sales_amount) desc
    limit 2
),
quarterly_growth as (
    select 
        qs.year,
        qs.quarter_id,
        qs.product_category_key,
        qs.english_product_category_name,
        qs.quarter_sales_amount,
        lag(qs.quarter_sales_amount) over (partition by qs.product_category_key order by qs.year, qs.quarter_id) as previous_quarter_sales
    from 
        quarterly_sales as qs
    join 
        top_categories as tc on qs.product_category_key = tc.product_category_key
)
select 
    year,
    quarter_id,
    product_category_key,
    english_product_category_name,
    quarter_sales_amount,
    round((quarter_sales_amount - previous_quarter_sales) * 100.0 / nullif(previous_quarter_sales, 0), 2) as quarter_over_quarter_growth_pct
from 
    quarterly_growth
order by 
    year, quarter_id, product_category_key;
"""

with connect_to() as pg:
    df_quarterly_growth = pd.read_sql(query_quarterly_growth, pg)
    df_quarterly_growth['year'] = df_quarterly_growth['year'].astype(int)
df_quarterly_growth


,year,quarter_id,product_category_key,english_product_category_name,quarter_sales_amount,quarter_over_quarter_growth_pct
0,2001,3.0,1,Bikes,1.453523e+06,NaN
1,2001,4.0,1,Bikes,1.812851e+06,24.72
2,2002,1.0,1,Bikes,1.791698e+06,-1.17
3,2002,2.0,1,Bikes,2.014012e+06,12.41
4,2002,3.0,1,Bikes,1.396834e+06,-30.64
5,2002,4.0,1,Bikes,1.327799e+06,-4.94
6,2003,1.0,1,Bikes,1.413530e+06,6.46
7,2003,2.0,1,Bikes,1.623971e+06,14.89
8,2003,3.0,1,Bikes,2.569678e+06,58.23
9,2003,3.0,4,Accessories,1.186745e+05,NaN


#### 2. **Сравнение будних и выходных дней**

In [161]:
query_weekday_vs_weekend_sales = """
set search_path to adv_works;
select 
    extract(year from s.order_date) as year,
    trim(to_char(s.order_date, 'Day')) as day_name,  -- Убираем лишние пробелы
    case 
        when extract(dow from s.order_date) in (0, 6) then 1  -- 0 = Sunday, 6 = Saturday
        else 0 
    end as is_weekend,
    round(sum(s.sales_amount), 2) as sales_amount
from 
    sales as s
group by 
    year, day_name, is_weekend
order by 
    year, is_weekend, day_name;
"""

with connect_to() as pg:
    df_weekday_vs_weekend_sales = pd.read_sql(query_weekday_vs_weekend_sales, pg)
df_weekday_vs_weekend_sales['year'] = df_weekday_vs_weekend_sales['year'].astype(int)
df_weekday_vs_weekend_sales


,year,day_name,is_weekend,sales_amount
0,2001,Friday,0,468724.50
1,2001,Monday,0,447197.10
2,2001,Thursday,0,450281.09
3,2001,Tuesday,0,433609.42
4,2001,Wednesday,0,435300.15
5,2001,Saturday,1,505234.58
6,2001,Sunday,1,526026.82
7,2002,Friday,0,856241.81
8,2002,Monday,0,919784.70
9,2002,Thursday,0,956084.69
